In [0]:
from keras.models import Sequential
import time

import seaborn as sns

import cv2
from skimage.transform import rotate, AffineTransform, warp
import os
from skimage import io, img_as_ubyte
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
import numpy.random as rng
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import random

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from PIL import Image
def load_training_data(DIR):
    IMG_SIZE=384
    train_data = []
    train_labels = []
    i = 0
    for img in os.listdir(DIR):
        i+=1
        path = os.path.join(DIR, img)
        name = img.split("_")[0] +" "+img.split("_")[1]
        img = Image.open(path)
        img = img.convert('RGB')
        img = img.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
        train_data.append(np.array(img)) 
        train_labels.append(name)
#         #########
        #Horizontal Flipping
        flip_img = Image.open(path)
        flip_img = flip_img.convert('RGB')
        flip_img = flip_img.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
        flip_img = np.array(flip_img)
        flip_img = np.fliplr(flip_img)
        train_data.append(np.array(flip_img))
        train_labels.append(name)
#         ##########
        # vertical flipping
        flip_img_up = Image.open(path)
        flip_img_up = flip_img_up.convert('RGB')
        flip_img_up = flip_img_up.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
        flip_img_up = np.array(flip_img_up)
        flip_img_up = np.flipud(flip_img_up)
        train_data.append(np.array(flip_img_up))
        train_labels.append(name)
        #########
       # positive rotation
        angle= random.randint(0,180)
        r1 = Image.open(path)
        r1= r1.convert('RGB')
        r1 = r1.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
        r1= np.array(r1)
        r1 = rotate(r1, angle)
        r1 = img_as_ubyte(r1) 
        train_data.append(r1)
        train_labels.append(name)
        # positive rotation
        angle= random.randint(0,180)
        r1 = Image.open(path)
        r1= r1.convert('RGB')
        r1 = r1.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
        r1= np.array(r1)
        r1 = rotate(r1, angle)
        r1 = img_as_ubyte(r1) 
        train_data.append(r1)
        train_labels.append(name)
        ##########
        # negative rotation
        angle= random.randint(0,180)
        r2 = Image.open(path)
        r2= r2.convert('RGB')
        r2 = r2.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
        r2= np.array(r2)
        r2 = rotate(r2, -angle)
        r2 = img_as_ubyte(r2) 
        train_data.append(r2)
        train_labels.append(name)
        #####
        # negative rotation
        angle= random.randint(0,180)
        r2 = Image.open(path)
        r2= r2.convert('RGB')
        r2 = r2.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
        r2= np.array(r2)
        r2 = rotate(r2, -angle)
        r2 = img_as_ubyte(r2) 
        train_data.append(r2)
        train_labels.append(name)
        # blur
        
        img2 = Image.open(path)
        img2 = img2.convert('RGB')
        img2 = img2.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
        img2 = np.array(img2)
        img2 =cv2.GaussianBlur(img2, (9,9),cv2.BORDER_DEFAULT)
        train_data.append(np.array(img2))
        train_labels.append(name)
        # add noise
        if i >= 1000:
            break
        ###
    return np.asarray(train_data),np.asarray(train_labels)

In [0]:
from PIL import Image
def load_testing_data(DIR):
    IMG_SIZE=384
    train_data = []
    train_labels = []
    i = 0
    for img in os.listdir(DIR):
        i+=1
        path = os.path.join(DIR, img)
        name = img.split("_")[0] +" "+img.split("_")[1]
        img = Image.open(path)
        img = img.convert('RGB')
        img = img.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
        train_data.append(np.array(img)) 
        train_labels.append(name)
    return np.asarray(train_data),np.asarray(train_labels)

In [0]:
from sklearn.model_selection import GridSearchCV 
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from joblib import dump, load
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

In [0]:
x_train, y_train = load_training_data("/content/drive/My Drive/cropped")

In [0]:
x_test, y_test = load_testing_data("/content/drive/My Drive/y_test_rita")

In [0]:
# plt.imshow(x_test[0])

In [0]:
# np.savetxt("/content/drive/My Drive/y_train_rita.txt", y_train, fmt='%s')
# np.savetxt("/content/drive/My Drive/y_train_rita_2nd_format.txt", y_train, newline=" ", fmt='%s')

In [0]:
# y_train

In [0]:
print(x_train.shape)
x_train=x_train[:1000,:]
print("xnew: ", x_train.shape)
x_train = x_train.reshape(1000,384*384*3)
print("xnewest: ", x_train.shape)
print(y_train.shape)
y_train = y_train[:1000]
print("ynew: ", y_train.shape)          

(6000, 384, 384, 3)
xnew:  (1000, 384, 384, 3)
xnewest:  (1000, 442368)
(6000,)
ynew:  (1000,)


In [0]:
print(x_test.shape)
x_test = np.reshape(x_test, (x_test.shape[0], -1))
print(x_test.shape)
print(y_test.shape)

(952, 384, 384, 3)
(952, 442368)
(952,)


In [0]:
print("final shapes x and y: ")
print(x_train.shape)
print(y_train.shape)
print("final test x and y shapes")
print(x_test.shape)
print(y_test.shape)

final shapes x and y: 
(1000, 442368)
(1000,)
final test x and y shapes
(952, 442368)
(952,)


In [0]:
def svm_linear(c):
    svc = SVC(probability = False, kernel = 'linear', C = c)
    
    svc.fit(x_train, y_train) 
    
  
    y_pred_train = svc.predict(x_train)
    acc_train = np.mean(y_pred_train == y_train)
    print('Train Accuracy = ', acc_train)

    y_pred_test = svc.predict(x_test)
    acc_test = np.mean(y_pred_test == y_test)
    print('Test Accuracy = ', acc_test)

    

In [0]:
svm_linear(0.1)



Train Accuracy =  1.0
Test Accuracy =  0.03361344537815126


In [0]:
def svm_rbf(c, g):
    svc_rbf = SVC(probability = False, kernel = 'rbf', C = c, gamma = g)
    
    # Fit the classifier on the training set.
    svc_rbf.fit(x_train, y_train) 
    y_pred_train = svc_rbf.predict(x_train)
    acc_train = np.mean(y_pred_train == y_train)
    print('Train Accuracy = ', acc_train)
    
    # Find the prediction and accuracy on the test set.
    y_pred_test = svc_rbf.predict(x_test)
    acc_test = np.mean(y_pred_test == y_test)
    print('Test Accuracy = ', acc_test)

In [0]:
svm_rbf(0.1, 'auto')

Train Accuracy =  0.09
Test Accuracy =  0.015756302521008403
